![](https://global-uploads.webflow.com/5f4444910aa0ad6a50bb4f52/5f444fb00e4dc15dd0f0416e_sisu-logo.svg)

# Task List Processing <span style = "color:#FF5000">Version 09 </span></div>

NOTES:

- if the users are not already in the system the matchup will not work. 
    - Need to set up an alert for when the users are not in the systen and initiate the user import. 

<div style = "text-align: left"> The purpose of this notebook is to process, and reformat,
                                   Task List information, <br> brought in as an excel document,
                                   down to SQL statements for inserting into <span style = "font-family:Roboto Thin; font-size:1em; color:#FF5000">sisu</span>. </div>

<a id='top_cell'></a>
[Change Log](#change_log)

## 0 <span style="color:#b54dff">Library

In [1]:
import os
from termcolor import colored
import pwd # needed if the document is stored on computer instead of Google Drive
import tkinter as tk
from tkinter import filedialog
import pandas as pd
pd.set_option('display.max_columns', None) # keeps pandas from truncating columns
import numpy as np
import clipboard
pd.options.display.max_colwidth = 1000
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
from datetime import datetime
import pytz
import task_list_tools as tlt


run_list = []

# Cell feedback
def cell_feedback():
    print("Cell ran at:")
    tz_east = pytz.timezone('US/Eastern') 
    datetime_eastern = datetime.now(tz_east)
    print("Eastern:", datetime_eastern.strftime("%H:%M:%S"))

    tz_central = pytz.timezone('US/Central') 
    datetime_central = datetime.now(tz_central)
    print("Central:", datetime_central.strftime("%H:%M:%S"))

    tz_pacific = pytz.timezone('US/Pacific')
    datetime_pacific = datetime.now(tz_pacific)
    print("Pacific:", datetime_pacific.strftime("%H:%M:%S"))


import subprocess
import platform

def raise_app(root: tk):
    root.attributes("-topmost", True)
    if platform.system() == 'Darwin':
        tmpl = 'tell application "System Events" to set frontmost of every process whose unix id is {} to true'
        script = tmpl.format(os.getpid())
        output = subprocess.check_call(['/usr/bin/osascript', '-e', script])
    root.after(0, lambda: root.attributes("-topmost", False))


## 1 <span style="color:#8b49fc"> Select File, View Information and Clean

<span style = "color:#73efff"> Make sure that the file you are working with is in a *FOLDER* that is formated with the team ID, a space, and then the team name. For example: 12345 Pretend Realty

In [3]:
'''
This cell collects the template file, does a few transformations, and validates the date. It will give Warnings and Errors if there is an concerne or issue with the data. 
'''

team_id, team_name, df = tlt.get_task_list_file_and_validate()

Team ID:	14778
Team Name:	Brownell Team Realtors
File Name:	14778 Task List 20220808.xlsx
Sheet 1 (i=0):   	START HERE => Enter your Users
Sheet 2 (i=1):   	Buyer Under Contract
Sheet 3 (i=2):   	Buyer New Home
Sheet 4 (i=3):   	Listing For Sale
Sheet 5 (i=4):   	Listing In Escrow
Sheet 6 (i=5):   	Master List (Make a copy)
Sheet 7 (i=6):   	Data mapping (hidden)
 
 
SHEETS WITH TASKS AND COUNT OF TASKS/ROWS:
31 rows in Buyer Under Contract
31 rows in Buyer New Home
31 rows in Listing For Sale
51 rows in Listing In Escrow
0 rows in Master List (Make a copy)
144 rows in total.
 
All empty Day values have been set to zero.
All whitespace cleared from string values.
 
 
GOOD: NO duplicates found in data.
 
select Name, Team_id, Status from team where Team_id = 14778; has been added to your clipboard
Paste into the Raw Data Tool to validate that we are working with the correct Team ID.
Verify that the Team ID matches the ticket in JIRA.
Cell ran at:
Eastern: 17:02:42
Central: 16:02:42
Paci

In [6]:
df_both = df[df['Applies to Buyer/Seller']== 'BOTH (B&S)']

In [8]:
df.loc[df['Applies to Buyer/Seller'] == 'BOTH (B&S)', 'Applies to Buyer/Seller'] = 'b'
df_both.loc[df_both['Applies to Buyer/Seller'] == 'BOTH (B&S)', 'Applies to Buyer/Seller'] = 's'
df['Buyer/Seller code'] = df['Applies to Buyer/Seller']
df_both['Buyer/Seller code'] = df_both['Applies to Buyer/Seller']
df = df.append(df_both)


In [9]:
df

,index,Task List Name,List Description,Applies to Buyer/Seller,Buyer/Seller code,Task Name,Task Description,Task or Notification?,"Assign to TC, Agent or assignee full name",Assign To T/A/Agent ID,Task Trigger date \n(Relative due date),Trigger Date DB (Sisu),Days,sheet_name,List descr. remaining\ncharacters,Task description remaining\ncharacters,Task name remaining\ncharacters
0,2,Buyer Under Contract,,s,s,Order Sign with Flyer Box (if applicable),,T,TC,T,Listing Date,listing_dt,0,Buyer Under Contract,255,255,214
1,3,Buyer Under Contract,,s,s,Send email to client to send to friends,,T,TC,T,Listing Date,listing_dt,0,Buyer Under Contract,255,255,216
2,4,Buyer Under Contract,,s,s,Update/upload new listing to Facebook - Add property to BTR/Social Medias,"Facebook, Instagram",T,TC,T,Listing Date,listing_dt,0,Buyer Under Contract,255,236,182
3,5,Buyer Under Contract,,s,s,Add listing enhancements to Realtor.com,,T,TC,T,Listing Date,listing_dt,0,Buyer Under Contract,255,255,216
4,6,Buyer Under Contract,,s,s,If property vacant - note to check property on Wednesday A or B,,N,TC,T,Listing Date,listing_dt,0,Buyer Under Contract,255,255,192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,49,Listing In Escrow,,s,s,Inform Team of Closing,,N,TC,T,Closed date,closed_dt,0,Listing In Escrow,255,255,233
144,50,Listing In Escrow,,s,s,Change MLS to S,,T,TC,T,Closed date,closed_dt,0,Listing In Escrow,255,255,240
145,51,Listing In Escrow,,s,s,Unassigning from lockbox in Sentrilock,,T,TC,T,Closed date,closed_dt,0,Listing In Escrow,255,255,217
146,52,Listing In Escrow,,s,s,Update file in SkySlope to - Under Contract,,T,TC,T,Acceptance Date,acceptance_dt,0,Listing In Escrow,255,255,212


## 2 <span style = "color:#577eff"> Task Lists </span> 

In [12]:
''' 
This cell produces query for retreiving the current Task Lists
The query text will automatically save to the clipboard, but in the case that the clipboard is cleared
the texted is also saved to the sql_get_task_lists variable. 
'''
sql_get_task_lists = tlt.get_task_lists(team_id)

Get current Task Lists SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
Cell ran at:
Eastern: 17:12:15
Central: 16:12:15
Pacific: 14:12:15


In [13]:
'''
Retrieve copied data from SQL output
Also define a Reset DataFrame
'''

df_reset_1, current_task_list_names = tlt.retrieve_current_task_lists_data(df)

'''
Getting information about the Task List count
'''
final_task_list_count = tlt.task_list_feedback(df, current_task_list_names)

TASK LIST COUNT
4 Current Task Lists
5 New Task Lists
9 Total Task Lists
Cell ran at:
Eastern: 17:12:32
Central: 16:12:32
Pacific: 14:12:32


In [16]:
'''
Adding missing columns to the DataFrame
'''

df, team_id = tlt.adding_columns(df, team_id)

Team ID column added.
created_ts column added.
updated_ts column added.
Status column added.
status_trigger column added.
Cell ran at:
Eastern: 17:13:34
Central: 16:13:34
Pacific: 14:13:34


In [17]:
'''
Define client task lists for INSERT statement
'''

df_client_task_list, client_task_list_cols_order = tlt.define_client_task_list(df, current_task_list_names, final_task_list_count)


'''
This cell produces an INSERT statement for the new Task Lists. 
The statment will automatically be saved to the clipboard but in the case that its deleted
the statement is saved to sql_insert_task_lists.capitalize

If you need to use sql_insert_task_lists, you can just type it in a code cell and it will output,
but the easiest way to use it is to type(copy) the following into a code cell:

clipboard.copy(sql_insert_task_lists)

'''

df, df_client_task_list, sql_insert_task_lists = tlt.insert_task_lists(df, df_client_task_list, team_id)

 
Inserting Task List from the Team 14778 template
The Task List INSERT SQL from the team 14778 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Lists from the template into Sisu.
Cell ran at:
Eastern: 17:13:42
Central: 16:13:42
Pacific: 14:13:42


## 3 <span style="color:#40a0bd"> Task Blueprints </span>  

In [18]:
'''
This cell produces an SQL query for getting the current task bluprints.
The query text will automatically be saved to the clipboard but if it is deleted
the query is also saved to the sql_get_task_blueprints variable. 
'''

sql_get_task_blueprints = tlt.get_task_blueprints(team_id)

Get Tasks SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
Cell ran at:
Eastern: 17:14:23
Central: 16:14:23
Pacific: 14:14:23


In [19]:
'''
This cell retrieves the current task blueprints.
Copy the queried values in the Raw Data Tool and run this cell.
'''

current_task_blueprint = tlt.retrieve_task_blueprints()

'''
Get feedback about task counts
'''

client_task_blueprint_cols, final_task_name_count, new_task_count = tlt.task_blueprint_feedback(df, current_task_blueprint)

TASK COUNT
151 Current Tasks
200 New Tasks
351 Total Tasks
Cell ran at:
Eastern: 17:14:36
Central: 16:14:36
Pacific: 14:14:36


In [20]:
'''
This cell saves a query to the clipboard for agent info. 
In case the data is deleted somehow, the data is saved to the sql_get_agent_info variable.
'''

sql_get_agent_info = tlt.get_agent_info(team_id)

Get Agent SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
Cell ran at:
Eastern: 17:14:42
Central: 16:14:42
Pacific: 14:14:42


In [21]:
df, df_reset_2, df_assign_map, df_assign_map_general, df_client_task_blueprints = tlt.process_agent_info(df, client_task_blueprint_cols)

sql_insert_task_blueprints = tlt.insert_task_blueprints(df_client_task_blueprints, team_id)

Inserting Task Blueprints from the Team 14778 template
An INSERT statement for the Task Blueprints from the team 14778 template has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Task Blueprints from the template into Sisu.
Cell ran at:
Eastern: 17:14:53
Central: 16:14:53
Pacific: 14:14:53


## 4 <span style="color:#44d47d"> Matchup

In [22]:
'''
This cell creates a query for the task list matchup data. 
This will save the text for the query to the clipboard. 
If the clipboard is cleared for some reason, use the sql_task_list_matchup variable
'''

sql_task_list_matchup = tlt.get_task_list_mathcup_data(team_id)

SQL query has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the following cell.
Cell ran at:
Eastern: 17:15:21
Central: 16:15:21
Pacific: 14:15:21


In [24]:
'''
This cell retrieves the task list data for the matchup
'''

df_matchup_task_lists = tlt.retrieve_task_list_matchup_data()

Cell ran at:
Eastern: 17:15:40
Central: 16:15:40
Pacific: 14:15:40


In [25]:
df_matchup_task_lists

,task_list_id,name,dscr,client_type_id
0,17180,Seller Contract to Close,NaN,s
1,17181,Listing Signed Task List,None,s
2,17182,Buyer Contract to Close,None,b
3,20084,ULRG New Team Member Checklist,None,NaN
4,28240,Buyer Under Contract,NaN,s
5,28241,Buyer New Home,NaN,b
6,28242,Listing For Sale,NaN,s
7,28243,Listing In Escrow,NaN,b
8,28244,Listing In Escrow,NaN,s


In [26]:
'''
This cell creates a query for the task blueprint matchup data. 
This will save the text for the query to the clipboard. 
If the clipboard is cleared for some reason, use the sql_task_blueprint_matchup variable 
'''

sql_task_blueprint_matchup_data = tlt.get_task_blueprint_matchup_data(team_id)

Get Tasks SQL has been copied to your clipboard. 
Paste this into Sisu's Raw Data Tool
Copy the returned table from the Raw Data Tool and run the next cell.
Cell ran at:
Eastern: 17:15:57
Central: 16:15:57
Pacific: 14:15:57


In [27]:
'''
This cell retrieves the task blueprint matchup data.
It also sets df_reset_3
'''

df_reset_3, df_matchup_task_blueprint = tlt.retrieve_task_blueprint_matchup_data(df)

Cell ran at:
Eastern: 17:16:09
Central: 16:16:09
Pacific: 14:16:09


### Checking Merges (Josh uses these for debugging purposes)

In [28]:
'''
Test merge
'''

tlt.test_merge(new_task_count, df_matchup_task_blueprint, df_matchup_task_lists, df)

Expected Number of Tasks: 200
Number of Tasks before removing duplicates:  200
Number of duplications:  1.0
Number of Tasks after removing duplicates:  200
Cell ran at:
Eastern: 17:16:16
Central: 16:16:16
Pacific: 14:16:16


### Resuming Merge

In [29]:
'''
Merge Data
'''

df, df_reset_4 = tlt.merge_data(df_matchup_task_blueprint, df, df_matchup_task_lists, new_task_count)

Number of Tasks before removing duplicates:  200
Number of duplications:  1.0
Cell ran at:
Eastern: 17:16:19
Central: 16:16:19
Pacific: 14:16:19


In [64]:
df_matchup_task_blueprint[df_matchup_task_blueprint['name'].str.contains('Deliver closing basket with keys')]

,task_blueprint_id,display_order,name,dscr,related_client_date_column,due_days,task_type,assign_to
208,266567,57,Deliver closing basket with keys,NaN,forecasted_closed_dt,0,T,NaN


In [65]:
df[df['task_blueprint_id'] == 266567]

,Task List Name,List Description,Applies to Buyer/Seller,Buyer/Seller code,Task Name,Task Description,Task or Notification?,"Assign to TC, Agent or assignee full name",Assign To T/A/Agent ID,Task Trigger date \n(Relative due date),Trigger Date DB (Sisu),Days,sheet_name,List descr. remaining\ncharacters,Task description remaining\ncharacters,Task name remaining\ncharacters,Team ID,created_ts,updated_ts,Status,status_trigger,first_name,last_name,agent_id,name_x,assign_to,display_order,client_type_id_x,name_y,dscr_x,client_type_id_y,name,dscr_y,related_client_date_column,due_days,task_type,task_list_id,task_blueprint_id


In [66]:
df.loc[df.index == 57, 'task_blueprint_id'] = 266567

In [67]:
df[df['task_blueprint_id'].isna()]

,Task List Name,List Description,Applies to Buyer/Seller,Buyer/Seller code,Task Name,Task Description,Task or Notification?,"Assign to TC, Agent or assignee full name",Assign To T/A/Agent ID,Task Trigger date \n(Relative due date),Trigger Date DB (Sisu),Days,sheet_name,List descr. remaining\ncharacters,Task description remaining\ncharacters,Task name remaining\ncharacters,Team ID,created_ts,updated_ts,Status,status_trigger,first_name,last_name,agent_id,name_x,assign_to,display_order,client_type_id_x,name_y,dscr_x,client_type_id_y,name,dscr_y,related_client_date_column,due_days,task_type,task_list_id,task_blueprint_id


In [68]:
'''
Validate Merge -- checks for Tasks that do not have task_blueprint_id
'''

tlt.validate_merge(df)

Cell ran at:
Eastern: 17:27:34
Central: 16:27:34
Pacific: 14:27:34


In [69]:
'''
Create INSERT statement for merge data.
This will automatically save to clipboard. If somehow deleted use:

clipboard.copy(sql_merge_data)
'''

sql_merge_data = tlt.create_merge_insert_statement(df, team_id)

Inserting Matchup data for Team 14778
An INSERT statement for the Matchup data for Team 14778 has been copied to your clipboard. 
Paste into the Sisu Raw Data Tool 
This will load the Matchup data into Sisu.
Cell ran at:
Eastern: 17:27:36
Central: 16:27:36
Pacific: 14:27:36


In [70]:
'''
This cell creates a summary. Copy the summary table and paste it into a markdown cell. Run the Markdown cell and copy the output. Paste the output into JIRA. 
'''

tlt.create_summary(current_task_list_names, current_task_blueprint, df)

SUMMARY
Copy the white text below and paste it into the following markdown cell. Run the markdown cell, select the output, copy it, and paste into JIRA.
| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       4 |
| New Task List Count     |       4 |
| Total Task List Count   |       8 |
| Initial Task Count      |     151 |
| New Task Count          |     200 |
| Total Task Count        |     351 |
Cell ran at:
Eastern: 17:27:48
Central: 16:27:48
Pacific: 14:27:48


| Subject                 |   Count |
|:------------------------|--------:|
| Initial Task List Count |       4 |
| New Task List Count     |       4 |
| Total Task List Count   |       8 |
| Initial Task Count      |     151 |
| New Task Count          |     200 |
| Total Task Count        |     351 |

## FINISH

#### Before you go:

If changes were made to this document, please fill out an entry in the change log. To do this:

* Copy the below template (inside the triple backticks) and paste above the last change log entry. *If you don't see the triple backticks, double-click this cell.*
* Fill in the missing information in the template with information about the changes. Details are appreciated. 



Change log template:
```
---

YYYY-MM-DD <br>
---

Comments and information about changes made. 

-Your name

---
```

# <span style="color:#568df5"> Change Log

<a id='change_log'></a>

[Back to Top](#top_cell)

---

2022-07-18 <br>
---

New special apostrophe character that had to be removed during validation. 
During the merge phase, after retrieving the data for the task lists and task blueprints, a ```fillna(")``` has to be used on the datasets
or the active columns might not match (that is, ```"" != NaN``` and ```NaN != NaN```)

-Josh Spradlin

---

---

2022-07-11 <br>
---

- Created a tools file where all the processes are now located to slim down the tool so that it would be easier to use.
- Added validation for Task Lists in data without trigger dates in the ```get_task_list_file_and_validate()``` function.
- Added feature that removes excess spece between agent names. 
- When calling information from Sisu, the Days values are formatted as integers, which cannot merge with the values in the data that are set as strings. 
    added command to change the feaure format to string. 


-Josh Spradlin

---

---

2022-06-21 <br>
---

For current template version, set the Buyer/Seller code values to be case insensitive.

Added function to remove duplicates during the matchup. 

```python
def merge_and_remove_duplicated_tasks():
    df_merge_check = df.merge(df_matchup_task_lists, left_on = ['Task List Name', 'List Description', 
                                                        'Buyer/Seller code'], 
                                                        right_on = ['name', 'dscr', 'client_type_id'] , 
                                                        how = 'left').merge(df_matchup_task_blueprint, 
                                                        left_on = ['Task Name','Trigger Date DB (Sisu)', 'Days', 
                                                        'Task or Notification?', 'assign_to', 'display_order'] , 
                                                        right_on = ['name', 'related_client_date_column', 
                                                        'due_days', 'task_type', 'assign_to', 'display_order'], 
                                                        how = 'left').drop('index', axis = 1)
    print("Number of Tasks before removing duplicates: ", len(df_merge_check))
    if len(df_merge_check)%new_task_count == 0:
        print("Number of duplications: ", len(df_merge_check)/new_task_count)
    
    merge_check_columns = df_merge_check.columns[~df_merge_check.columns.str.contains(
                                                                'task_list_id|task_blueprint_id')]
    return df_merge_check[merge_check_columns].drop_duplicates(keep='last').merge(
            df_merge_check[['task_list_id', 'task_blueprint_id']], left_index = True, right_index = True)

```

-Josh Spradlin

---

---

2022-06-20 <br>
---
Updated tkinter options so that the file dialog will open as the front most window. 

&
 
Created function to give the time that each cell is ran for feedback purposes, and added to each cell. 
```python
# Cell feedback
def cell_feedback():
    print("Cell ran at:")
    tz_east = pytz.timezone('US/Eastern') 
    datetime_eastern = datetime.now(tz_east)
    print("Eastern:", datetime_eastern.strftime("%H:%M:%S"))

    tz_central = pytz.timezone('US/Central') 
    datetime_central = datetime.now(tz_central)
    print("Central:", datetime_central.strftime("%H:%M:%S"))

    tz_pacific = pytz.timezone('US/Pacific')
    datetime_pacific = datetime.now(tz_pacific)
    print("Pacific:", datetime_pacific.strftime("%H:%M:%S"))
```
-Josh Spradlin

---



---

2022-06-14 <br>
---


Added validations for **Buyer Seller code** and <b>Task or Notification?</b> columns to the "FILE" cell:

```python
# Validate Buyer/Seller code column
if len(df[df['Buyer/Seller code'].str.contains("b|s")]) == len(df):
    pass
else:
    print(colored("WARNING:", 'red', attrs = ['bold']) +  f" {len(df) - len(df[df['Buyer/Seller code'].str.contains('b|s')])} Buyer/Seller code contains incorrect characters.")
    print(" ")

# Validate Task or Notification column
if len(df[df['Task or Notification?'].str.contains("T|N")]) == len(df):
    pass
else:
    print(colored("WARNING:", 'red', attrs = ['bold']) +  f" {len(df) - len(df[df['Task or Notification?'].str.contains('b|s')])} Task or Notification? contains incorrect characters.")
    print(" ")

```

-Josh Spradlin

---
    

---

2022-06-10 <br>
---

Added following code to the 'FILE' cell (line 180)

```python 
df['List Description'] = df['List Description'].fillna('')
```

If there are not List descriptions, the forward fill from the previous line will fill in everything with NaN which cannot be matched in the matchup portion of the notebook. 

<span style = "color:magenta"> <b>THIS IS A BANDAID!!</b></span> If there is a list description for one list, but not for the following, the code will forward fill (impute) the first list description until there is another list description (or until the end of the data).

<span style="color:cyan">Planning on making this something that works uniquely with Task List Names.

Something like this
```python
for i in df['Task List Name'].unique():
    df_TLD = df[df['Task List Name'] == i].reset_index(drop=True)
    if df_TLD['List Description'][0].notna():
        df_TLD['List Description'] = df_TLD['List Description'][0]
    else:
        df_TDL['List Description'] = df_TDL['List Description'].fillna('')
    df.loc[df['Task List Name'] == i, 'List Description'] = df_TDL['List Description'] 
```

<span style = "color:green">Saving as version 4


-Josh Spradlin 

---


---

2022-06-08 <br>
---
Edited the first SQL (Checking Teams ID and name), added trailing ";"
Comments and information about changes made. 

-Yves Robinet

---

---

2022-06-06 <br>
---

Changed text output for the 'Summary' cell to explain how to use the output table more clearly.

-Josh Spradlin

---

---

2022-06-01 <br>
---

Was having an issue with the matchup.
When using the select statements, we are not given any information that uniquely links the ```task_list_id``` to the ```task_blueprint_id```. 

My current solution is to use as many parameters available to get unique matches. **This can be tricked!!!!!** If the details of two or more ```Task``` are all exactly the same, but one is on another ```Task List``` AND they have the same ```display_order```, then the ```Task``` you are trying to link to the ```Task List``` and the identical task from the other Task List will both be matched to the Task List. This doesn't remove the Task from the other Task List. I don't know if this can cause issues, but I guess it's possible that this might cause double notifications.

-Josh Spradlin

---
